# Test full calculation
Test the full calculation of the 

In [1]:
pip install xmip

Note: you may need to restart the kernel to use updated packages.


## Import packages

In [2]:
%matplotlib inline
import xarray as xr
xr.set_options(display_style='html')
from xmip.preprocessing import rename_cmip6

import util

## Summary of some test data

| model name | resolution | years in file for one variable | file size (one variable) | combined 3D variable data size | combined 2D variable data size |
| --- | --- | --- | --- | --- | --- |
| ACCESS-OM2 | 100 km | 10 | 670 MB | 17 GB | 0.4 GB |
| ACCESS-OM2-025 | 25 km | 1 | 1.1 GB | 271 GB | 7 GB |
| CMCC-CM2-SR5 | 100 km | 61 | 3.6 GB | 15 GB | 0.4 GB |
| CMCC-CM2-HR4 | 25 km | 10 | 9.4 GB | 231 GB | 6 GB |

## Load data from ESGF

In [3]:
result = util.esgf_search(activity_id='OMIP', experiment_id='omip2', variable_id='vmo', source_id='CMCC-CM2-SR5')

https://esgf-node.llnl.gov/esg-search/search/?activity_id=OMIP&experiment_id=omip2&variable_id=vmo&source_id=CMCC-CM2-SR5&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


In [4]:
# pick one file for testing
files_to_open = [result[-1]]

In [5]:
ds = xr.open_mfdataset(files_to_open, combine='by_coords')

## Pre-proc with `XMIP`

In [6]:
ds = rename_cmip6(ds)

## Subset Pan-Arctic (lat>50N)

In [7]:
cond = (ds['lat']<=50)
dsnow = ds[['lat','lon','vmo']].where(cond,drop=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [8]:
dsnow

<xarray.Dataset>
Dimensions:  (y: 289, x: 360, time: 732, lev: 50)
Coordinates:
  * time     (time) object 1958-01-16 12:00:00 ... 2018-12-16 12:00:00
  * lev      (lev) float64 0.5126 1.621 2.858 ... 5.1e+03 5.498e+03 5.904e+03
  * y        (y) int32 0 1 2 3 4 5 6 7 8 ... 280 281 282 283 284 285 286 287 288
  * x        (x) int32 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359
Data variables:
    lat      (y, x) float64 dask.array<chunksize=(289, 360), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(289, 360), meta=np.ndarray>
    vmo      (time, lev, y, x) float32 dask.array<chunksize=(732, 50, 289, 360), meta=np.ndarray>
Attributes: (12/39)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     OMIP
    comment:                         Ocean initial conditions: WOA 2013 T & S...
    contact:                         Pier Giuseppe Fogli (piergiuseppe.fogli@...
    creation_date:                   2020-02-19T13:20:13Z
    data_specs_version:              01.00.31
    ...                              ...
    variable_id:                     vmo
    variant_label:                   r1i1p1f1
    license:                         CMIP6 model data produced by CMCC is lic...
    cmor_version:                    3.5.0
    tracking_id:                     hdl:21.14100/119a9ea0-a62e-46ac-804d-879...
    DODS_EXTRA.Unlimited_Dimension:  time

## Extract Berring Strait

## Calc Volume Transport